In [ ]:
! pip install bs4 requests pandas

In [40]:
def build_player(row):    
    cells = row.find_all('td')
    name_link = cells[1].find('a')
    
    if name_link is None:
        return None

    rank = cells[0].text.strip()
        
    name = name_link.text.strip()
    name_url = name_link['href']
    team_link = cells[2].find('a')
    team = team_link.text.strip()
    team_url = team_link['href']
    salary = cells[3].text.strip()
    
    return {
        'rank': rank,
        'name': name,
        'name_url': name_url,
        'team': team,
        'team_url': team_url,
        'salary': salary
    }    

In [ ]:
import requests
from bs4 import BeautifulSoup


def crawler(page=1):
    url = f"https://www.espn.com/nba/salaries/_/year/2023/page/{page}/seasontype/4"

    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')

    page = soup.find(class_='page-numbers')
    values = page.text.split('of')
    currentPage = values[0]
    pageNumber = values[1]

    return soup, int(currentPage), int(pageNumber)


soup, currentPage, pageNumber = crawler()


players = []
for i in range(currentPage+1, pageNumber):
    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows:
        player = build_player(row)
        if player is not None:
            players.append(player)

    soup, currentPage, pageNumber = crawler(i)            


In [50]:
import pandas as pd

df = pd.DataFrame(data=players)

df.to_csv('../dataset/scraping/players.csv', index=False)